 ________
<font size=6, color=blue>  Venue Recommendation Engine : </font>

### -- Suggest Venues based on previous venues liked by a user on FourSquare 
 ________


Nowadays, travel and exploration are a part of everyday life. People find themselves exploring new areas of their towns, cities, countries or even the world. This recommendation engine aims to make it easier for the wanderlusts by recommending places that they could like in any area of the world based on what they have liked earlier


<font color='blue'> Importing Libraries required for data wrangling, processing, recommendation and visualization </font>

In [2]:
## Uncomment the installation line if the packages are not installed
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab


import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


___
### __**Part 1**__ : Read Data from FourSquare to understand what a User Likes
___ 

#### _Define Foursquare Credentials and Version_

In [3]:
CLIENT_ID = 'DEVKHI1LOIIJM5Y5Y1JW0CNYR320OBTTOM3AH3XKRTNC0BDW' # your Foursquare ID
CLIENT_SECRET = 'BJHLXVSUKB2CABIFLCTMXCU1QOVD03AT3YOQLHIO1BFNYE4K' # your Foursquare Secret
VERSION = '20200601' # Foursquare API version
ACCESS_TOKEN = 'T5QHM2HWIB2HGR0R3OASHFQH4VQB3MPLJMVBDOAVCB2WZWO0'

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

#### _Set the User ID for for whom recommendations are being made_

In [24]:
# Set the user ID to a FourSquare User who has a list of Liked Venues
#user_id = '10447572'
user_id = '51871889'


#### _Get the List ID for the list of Liked Places from FourSquare_

In [25]:
## Use the Users->lists Endpoint to retrieve the List ID
limit = 1

url = 'https://api.foursquare.com/v2/users/{}/lists?client_id={}&client_secret={}&v={}&limit={}&oauth_token={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit,ACCESS_TOKEN)
results = requests.get(url).json()

## if the user has a list of Liked Places; retrieve the List ID
lists = results['response']['lists']['groups'][0]['items']
df = json_normalize(lists)
list_id = df['id'][df.type == 'liked'].tolist()[0]

print("The List Id that holds the list of liked places is:  {}".format(list_id))

The List Id that holds the list of liked places is:  51871889/venuelikes


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


#### _Get the Venues in the list of Liked Places from FourSquare_

<font size=2, color=grey>   The List Details End Point helps retrieve the Venues that the user has liked </font>

In [26]:
limit = 100

url = 'https://api.foursquare.com/v2/lists/{}?client_id={}&client_secret={}&v={}&limit={}&oauth_token={}'.format(list_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit,ACCESS_TOKEN)
results = requests.get(url).json()

#### _Create a Dataframe of the Liked Venues_

In [27]:
venues = results['response']['list']['listItems']['items']
#venues = results['response']['items']['venues']
venues_list =[]
venues_list.append([(
            user_id, 
            v['user']['firstName'],
            v['user']['lastName'],
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])

liked_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
liked_venues.columns = ['UserId', 
                  'UserFName', 
                  'UserLName',
                  'VenueId',      
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

liked_venues = liked_venues[liked_venues['Venue Category'] != 'Neighborhood']

In [28]:
liked_venues.head()

,UserId,UserFName,UserLName,VenueId,Venue,Venue Latitude,Venue Longitude,Venue Category
0,51871889,Amelia,Mani,57e0745a498ea809dbf75f68,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,51871889,Amelia,Mani,5a4fdf56772fbc5e9fa73c7f,Chotto Matte,43.646473,-79.378782,Japanese Restaurant
2,51871889,Amelia,Mani,5a273990a423620ac8bfe98d,Hawking Bird,37.835875,-122.262844,Thai Restaurant
3,51871889,Amelia,Mani,5c48ef89a6ec98002c7d63cd,Beit Rima,37.768754,-122.429040,Middle Eastern Restaurant
4,51871889,Amelia,Mani,4cfa7e18d8468cfa612aee6b,El Dollar Taqueria,35.614588,-119.657344,Taco Place


___
### __**Part 2**__ : Get the Venue Details of the area that needs to be explored
___

#### _Define Function to retrieve Nearby Venues in a Specific Area_

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("Retrieving the venue details for: {}".format(name))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'VenueId',           
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### _Decide the Area to be explored and get its geo-location_

In [43]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


#### _Retrieve Venues in the area of Interest_

In [44]:
# type your answer here
LIMIT = 100
radius = 1000

test_venues = getNearbyVenues(names=[address],
                                   latitudes=[latitude],
                                   longitudes=[longitude]
                                  )

test_venues = test_venues[test_venues['Venue Category'] != 'Neighborhood']

Retrieving the venue details for: Toronto, ON


#### _Check if sufficient Venues are available in the area of Interest_

In [45]:
print("There are {} venues in {} that will be analyzed".format(test_venues.shape[0], address))

There are 76 venues in Toronto, ON that will be analyzed


___
### __**Part 3**__ : Prepare the Datasets for Recommendation Analysis
___

##### **The following steps are being followed for scoring the target venues**
 - <font size=2> Create a combined dataset of all venues </font>
  - <font size=2>Liked Venues by User</font> 
  - <font size=2>Target Venues to score/rank </font> <br>
This is necessary so that the on-hot encoded matrix for the Target Venues can be appropriately multiplied by the User Profile created
 - <font size=2> Use the Unique Categories for one hot encoding by Venue Category. </font>   
 - <font size=2>Create a User Profile for the user by using one-hot encoding data for Venue Category and summing up the values for all rows </font>
 - <font size=2>Create a matrix of features for the target venues to rank by using one hot encoding for Venue Category </font>
- <font size=2>Create a recommendation table for the Venues by multiplying the feature matrix by the user profile </font>
- <font size=2>Drop all records that have a score of Zero </font>

  
  





#### _Combine Venue data to get a common set of Categories_
   --- This is necessary to get a common set of categories so that the features of the user profile and the test data can be correctly compared   

   <font color='blue'> Set a specific identifier for the Venue data to be explored so that it can separated later  </font> 

In [46]:
test_venues['UserId'] = "TESTDATA"

   <font color='blue'> Combine the liked Venues and the Venues to be explored   </font> 

In [47]:
combined_data = liked_venues[['UserId','VenueId','Venue Category']]

combined_data = combined_data.append(test_venues[['UserId','VenueId','Venue Category']])
combined_data.reset_index(drop=True, inplace=True)

print("No of Records in Liked Venues : {}".format(liked_venues.shape[0]))
print("No of Records in Test Venues : {}".format(test_venues.shape[0]))
print("No of Records Together : {}".format(combined_data.shape[0]))



print("Unique Categories in the Liked Venues : {}".format(len(liked_venues['Venue Category'].unique())))
print("Unique Categories in the Test Venues : {}".format(len(test_venues['Venue Category'].unique())))
print("Unique Categories in the Combined List: {}".format(len(combined_data['Venue Category'].unique())))



No of Records in Liked Venues : 99
No of Records in Test Venues : 76
No of Records Together : 175
Unique Categories in the Liked Venues : 57
Unique Categories in the Test Venues : 56
Unique Categories in the Combined List: 99


#### _One Hot Encoding of the venue data based on all the categories_ 

In [48]:
combined_onehot = pd.get_dummies(combined_data[['Venue Category']], prefix="", prefix_sep="")

## Insert the VenueID and the UserID columns so that the One-Hot Encoded Data can be joined to the Venue Data
combined_onehot.insert(0, 'VenueId', combined_data['VenueId'])
combined_onehot.insert(0, 'UserId', combined_data['UserId'])

#### _Merge the One Hot Encoded Data with the Venue Data_  

In [49]:
## Use the VenueID and UserID columns for making sure that the data is joined correctly
test_venues_merged = test_venues
#print(test_venues_merged.shape)
test_venues_merged = pd.merge(test_venues_merged, combined_onehot, on=['UserId','VenueId'])

#### _Create User Profile based on liked Venues_ 

In [50]:
liked_venues_merged = liked_venues
#print(liked_venues_merged.shape)

liked_venues_merged = pd.merge(liked_venues_merged, combined_onehot, on=['UserId','VenueId'], how='left')
#print(liked_venues_merged.shape)

user_profile = liked_venues_merged.groupby('UserId').sum().reset_index()


#### _Drop the unnecessary columns from the user profile_

In [51]:
user_profile.drop(labels=['UserId','Venue Latitude','Venue Longitude'], inplace=True, axis=1)
userProfile = user_profile.transpose()
userProfile.columns = ['Value']
userProfile

,Value
Airport Lounge,2
American Restaurant,1
Art Gallery,1
Art Museum,0
BBQ Joint,1
Bagel Shop,1
Bakery,1
Bank,0
Bar,2
Beach,2


___
### __**Part 4**__ : Content Based Recommendation Calcualtion
___

#### _Create the Category Table from the Venue Data to be ranked_  

In [52]:
#The CategoryTable has the one-hot encoded category data for all categories

#Set the VenueID as the Index for the The CategoryTable
CategoryTable = test_venues_merged.set_index(test_venues_merged['VenueId'])

#And drop the unnecessary information
CategoryTable = CategoryTable.drop('Neighborhood', 1).drop('Neighborhood Latitude', 1).drop('Neighborhood Longitude', 1).drop('Venue', 1)
CategoryTable = CategoryTable.drop('Venue Latitude', 1).drop('Venue Longitude', 1).drop('Venue Category', 1).drop('UserId', 1)
CategoryTable = CategoryTable.drop('VenueId', 1)
CategoryTable.head(5)

,Airport Lounge,American Restaurant,Art Gallery,Art Museum,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bookstore,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Business Center,Café,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Department Store,Dessert Shop,Diner,EV Charging Station,Fast Food Restaurant,Food,Food & Drink Shop,Food Stand,Food Truck,Furniture / Home Store,Gastropub,General Travel,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mountain,Movie Theater,Museum,Music Venue,National Park,New American Restaurant,North Indian Restaurant,Pharmacy,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfall,Waterfront,Wine Bar,Winery,Women's Store,Zoo Exhibit
VenueId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4ad4c05ef964a520a6f620e3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4e2284b11fc7c0ef9857d143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4ad4c05ef964a520e2f620e3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4b2a6eb8f964a52012a924e3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4ae7b27df964a52068ad21e3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### _Calculate the rank for each Venue using Category Data_  

In [53]:
# Multiply the category data by the weights in the User Profile and calculate the Weighted Average
recommendationTable = ((CategoryTable*userProfile['Value']).sum(axis=1))/(userProfile['Value'].sum())

# Sort the recommendations based on the score in descending order
recommendationTable = recommendationTable.sort_values(ascending=False)

# Remove all venue ids from the list where the score = 0
recommendationTable = recommendationTable.loc[lambda x : x>0]
#print(recommendationTable)

# Limit Recommendations to 10
noRecs = len(recommendationTable)
if noRecs > 10:
    noRecs=10

recommendationTable.rename("Score", inplace=True)

    
recommendationTable

VenueId
4adc9148f964a520512d21e3    0.090909
4b5219ccf964a520d56727e3    0.030303
4b7aa994f964a5200c362fe3    0.030303
54132b3b498ee9ca9332e189    0.030303
4ad9ffbbf964a520091d21e3    0.030303
5a3a846af62e0960e9364d11    0.030303
4b50d070f964a5206d3327e3    0.030303
4dfe1cf0a809d61e2fc568ce    0.030303
4ad69511f964a520e40721e3    0.030303
4ad4c061f964a52095f720e3    0.030303
4d2c55ea55a8b60c5004efc0    0.030303
4e9de9fbb803b7506dcf4c41    0.030303
4ae60299f964a52003a421e3    0.030303
5773f01f498e98371390bdfd    0.020202
5a62412b1108ba191da5ba06    0.020202
5894c1f9266c1121f0a757d1    0.020202
4c9cd4d37c096dcb9034c5d1    0.020202
506db1a9e4b0a3f3b31412f0    0.010101
55074b57498ef35eeeaafe29    0.010101
4ad7929cf964a520500c21e3    0.010101
4ada04eff964a520571d21e3    0.010101
4f513029e4b07c3382c9fdb9    0.010101
50ecb1fae4b0beb13294f0aa    0.010101
5214e7c111d2a83379eae21f    0.010101
Name: Score, dtype: float64

___
### __**Part 5**__ : Displaying and Visualizing Results
___

#### _Displaying the Top 10 Recommendations_

In [54]:
#Creating the final recommendation table
rec_venues = test_venues

#Set the Index on VenueID to merge using Index; This eliminates Venues that do not have a recommendation score
rec_venues = pd.merge(rec_venues, recommendationTable, how='inner', left_on='VenueId', right_index=True)

#Drop all the Unnecessary Columns from the Recommended Venues Table
rec_venues.drop(labels =['UserId','Neighborhood Latitude','Neighborhood Longitude'], inplace=True, axis=1)

#Sort the Recommended Venues from the highest score to the lowest score
rec_venues.sort_values(by='Score',inplace=True, ascending=False)
rec_venues.reset_index(inplace=True, drop=True)
rec_venues.head(noRecs)

,Neighborhood,VenueId,Venue,Venue Latitude,Venue Longitude,Venue Category,Score
0,"Toronto, ON",4adc9148f964a520512d21e3,Chipotle Mexican Grill,43.656860,-79.380910,Mexican Restaurant,0.090909
1,"Toronto, ON",54132b3b498ee9ca9332e189,M Square Coffee Co,43.651218,-79.383555,Coffee Shop,0.030303
2,"Toronto, ON",4d2c55ea55a8b60c5004efc0,deKEFIR,43.650433,-79.379393,Ice Cream Shop,0.030303
3,"Toronto, ON",4b7aa994f964a5200c362fe3,Tim Hortons,43.655212,-79.380063,Coffee Shop,0.030303
4,"Toronto, ON",4ad9ffbbf964a520091d21e3,Jack Astor's Bar & Grill,43.656019,-79.380326,Restaurant,0.030303
5,"Toronto, ON",4b50d070f964a5206d3327e3,Thai Island,43.649146,-79.383798,Thai Restaurant,0.030303
6,"Toronto, ON",4e9de9fbb803b7506dcf4c41,Starbucks,43.654465,-79.378919,Coffee Shop,0.030303
7,"Toronto, ON",4ae60299f964a52003a421e3,Starbucks,43.650751,-79.388047,Coffee Shop,0.030303
8,"Toronto, ON",4ad4c061f964a52095f720e3,Salad King,43.657601,-79.381620,Thai Restaurant,0.030303
9,"Toronto, ON",4ad69511f964a520e40721e3,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant,0.030303


#### Create a map of the Recommended Locations
   - Marker for Rank 1 is displayed in red
   - Marker for Rank 2 is displayed in green
   - Marker for Rank 3 is displayed in blue
   - All other ranks are displayed in cyan
   

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)
# set color scheme for the clusters
x = np.arange(noRecs)
#ys = [i + x + (i*x)**2 for i in range(noRecs)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

rainbow =['red', 'green', 'blue', 'cyan' ]

# add markers to the map
markers_colors = []
rank=0
ColorIdx=0
for lat, lon, poi, cat in zip(rec_venues['Venue Latitude'], rec_venues['Venue Longitude'], rec_venues['Venue'], rec_venues['Venue Category']):
    rank=rank+1
    if rank <= 4:
        ColorIdx = ColorIdx + 1
    elif rank >= 11:
        break
        
    
    label = folium.Popup(' Venue: ' + str(poi) +'/ Category: ' + str(cat) + '/ Rank: ' + str(rank), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[ColorIdx-1],
        fill=True,
        fill_color=rainbow[ColorIdx-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

_____